<h2>Setting Stuff Up</h2>

Here we import some packages that we'll need in various places. We'll also load all the variables we set in config.

In [ ]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
loadvar()

## FOLLOWING ALONG AT HOME  

If you are following along at home using the docker-compose stack, you will need to run the following cell to get the hsotname and port of your tcp tunnel so Agave can contact your system without a public IP address.

In [ ]:
if os.environ.get('USE_TUNNEL') == 'True': 
    # fetch the hostname and port of the reverse tunnel running in the sandbox 
    # so Agave can connect to our local sandbox
    !echo $(ssh -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'') > ngrok_url.txt  
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#' > ngrok_port.txt

    # set the environment variables otherwise set when running in a training cluster
    os.environ['MACHINE_IP'] = readfile('ngrok_host.txt').strip()
    os.environ['MACHINE_PORT'] = readfile('ngrok_port.txt').strip()
    os.environ['AGAVE_SYSTEM_HOST'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_PORT'] = readfile('ngrok_port.txt').strip()


## Creating the Storage Machine   

Agave wants to know which place (or places) you want to store the data associated with your jobs. Here, we're going to set that up. Authentication to the storage machine will be through SSH keys. The key and public key files, however, contain newlines. To encode them in Json (the data format used by Agave), we will run the jsonpki command on each file. Next, we will store its contents in the environment for use by setvar.

In [ ]:
!jsonpki --public ~/.ssh/id_rsa.pub > ~/.ssh/id_rsa.pub.txt
!jsonpki --private ~/.ssh/id_rsa > ~/.ssh/id_rsa.txt

In [ ]:
os.environ["PUB_KEY"]=readfile("${HOME}/.ssh/id_rsa.pub.txt").strip()
os.environ["PRIV_KEY"]=readfile("${HOME}/.ssh/id_rsa.txt").strip()

In this next cell, we create the json file used to describe the storage machine.

In [ ]:
writefile("${AGAVE_STORAGE_SYSTEM_ID}.txt","""{
    "id": "${AGAVE_STORAGE_SYSTEM_ID}",
    "name": "${MACHINE_NAME} storage (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "type": "STORAGE",
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    }
}
""")

Here, we tell Agave about the machine. You can re-run the previous cell and the next one if you want to change the definition of your storage machine.

In [ ]:
!systems-addupdate -F ${AGAVE_STORAGE_SYSTEM_ID}.txt

Next we run the Agave command `files-list`. This provides a check that we've set up the storage machine correctly.

In [ ]:
!files-list -S ${AGAVE_STORAGE_SYSTEM_ID} ./ | head -5

<h2>Setting up the Execution Machine</h2>

You may not always wish to store your data on the same machine you run your jobs on. However, in this tutorial, we will assume that you do. The description for the execution machine is much like the storage machine. However, there are a few more pieces of information you'll need to provide. In this example, we are going to call commands directly on the host as opposed to using a batch queue scheduler. It is slightly simpler.

In [ ]:
# Edit any parts of this file that you know need to be changed for your machine.
writefile("${AGAVE_EXECUTION_SYSTEM_ID}.txt","""
{
    "id": "${AGAVE_EXECUTION_SYSTEM_ID}",
    "name": "${MACHINE_NAME} (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "public": false,
    "status": "UP",
    "type": "EXECUTION",
    "executionType": "CLI",
    "scheduler" : "FORK",
    "environment": null,
    "scratchDir" : "${SCRATCH_DIR}",
    "queues": [
        {
            "name": "none",
            "default": true,
            "maxJobs": 10,
            "maxUserJobs": 10,
            "maxNodes": 6,
            "maxProcessorsPerNode": 6,
            "minProcessorsPerNode": 1,
            "maxRequestedTime": "00:30:00"
        }
    ],
    "login": {
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        },
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SSH"
    },
    "maxSystemJobs": 50,
    "maxSystemJobsPerUser": 50,
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    },
    "workDir": "${AGAVE_STORAGE_WORK_DIR}"
}""")

In [ ]:
!systems-addupdate -F ${AGAVE_EXECUTION_SYSTEM_ID}.txt

In [ ]:
# Test to see if this worked...
!files-list -S ${AGAVE_EXECUTION_SYSTEM_ID} ./ | head -5

<h3>Create the Application</h3>
Agave allows us to describe custom allocations, limiting users to run a specific job. In this case, we're going to create a simple "fork" scheduler that just takes the command we want to run as a job parameter. The wrapper file is a shell script we will run on the execution machine. If we were using a scheduler, this would be our batch file.

In [ ]:
writefile("fork-wrapper.txt","""
#!/bin/bash
\${command}
""")

Using Agave commands, we make a directory on the storage server an deploy our wrapper file there.

In [ ]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N ${AGAVE_APP_DEPLOYMENT_PATH}
!files-upload -F fork-wrapper.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}/

All agave applications require a test file. The test file is a free form text file which allows you to specify what resources you might need to test your application.

In [ ]:
writefile("fork-test.txt","""
command=date
fork-wrapper.txt
""")

In [ ]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N ${AGAVE_APP_DEPLOYMENT_PATH}
!files-upload -F fork-test.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}/

Like everything else in Agave, we describe our application with Json. We specifiy which machines the application will use, what method it will use for submitting jobs, job parameters and files, etc.

In [ ]:
writefile("fork-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-fork",
   "version":"1.0",
   "label":"Runs a command",
   "shortDescription":"Runs a command",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"${AGAVE_APP_DEPLOYMENT_PATH}",
   "templatePath":"fork-wrapper.txt",
   "testPath":"fork-test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[
         {   
         "id":"datafile",
         "details":{  
            "label":"Data file",
            "description":"",
            "argument":null,
            "showArgument":false
         },
         "value":{  
            "default":"/dev/null",
            "order":0,
            "required":false,
            "validator":"",
            "visible":true
         }
      }   
   ],
   "parameters":[{
     "id" : "command",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"/bin/date",
       "validator":null
     },
     "details":{
         "label": "Command to run",
         "description": "This is the actual command you want to run. ex. df -h -d 1",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "label": "Command to run",
         "description": "This is the actual command you want to run. ex. df -h -d 1",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")

In [ ]:
!apps-addupdate -F fork-app.txt

<h2>Running Jobs</h2>
Now that we have specified our application using Agave, it is time to try running jobs. To start a job we, once again, create a Json file. The Json file describes the app, what resource to run on, as well as how and when to send notifications. Notifications are delivered by callback url. EMAIL is the easiest type to configure, but we show here how to send webhook notifications to the popular [RequestBin](https://requestb.in/). 

Before we configure our notification, we need to create a requestbin to use. There are convenience commands to interact with requestbin built into the Agave CLI. We will use those to get our URL.

In [ ]:
rburl = !requestbin-create 
os.environ['REQUESTBIN_URL'] = rburl[0].strip()

Now that we have a URL to recieve webhooks from our job, Let's look at our job request. The way this job is configured, it will send the requestbin notifications for every job event until the job reaches a terminal state. For a full list of job events, please see http://docs.agaveplatform.org/#job-monitoring

In [ ]:
writefile("job.txt","""
 {
   "name":"fork-command-1",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0",
   "executionSystem": "${AGAVE_EXECUTION_SYSTEM_ID}",
   "archive": false,
   "notifications": [
    {
      "url":"${REQUESTBIN_URL}?event=\${EVENT}&jobid=\${JOB_ID}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "command":"echo hello"
   }
 }
""")

Because the setvar() command can evalute `$()` style bash shell substitutions, we will use it to submit our job. This will capture the output of the submit command, and allow us to parse it for the JOB_ID. We'll use the JOB_ID in several subsequent steps.

In [ ]:
setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F job.txt)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")

<h2>Job Monitoring and Output</h2>

While the job is running, the requestbin you registered will receive webhooks from Agave every time a job event occurs. To monitor this in real time, evaluate the next cell an visit the printed url in your browser:

In [ ]:
print ('%s?inspect'%os.environ['REQUESTBIN_URL'])

Of course, you can also monitor the job status by polling. Note that the notifications you receive via email and webhook are less wasteful of resources. However, we show you this for completeness.

In [ ]:
for iter in range(20):
    setvar("STAT=$(jobs-status $JOB_ID)")
    stat = os.environ["STAT"]
    sleep(5.0)
    if stat == "FINISHED" or stat == "FAILED":
        break

The jobs-history command provides you a record of the steps of what your job did. If your job fails for some reason, this is your best diagnostic.

In [ ]:
!echo jobs-history ${JOB_ID}
!jobs-history ${JOB_ID}

This command shows you the job id's and status of the last 5 jobs you ran.

In [ ]:
!jobs-list -l 5

This next command provides you with a list of all the files generated by your job. You can use it to figure out which files you want to retrieve with jobs-output-get.

In [ ]:
!jobs-output-list --rich --filter=type,length,name ${JOB_ID}

Retrieve the standard output.

In [ ]:
!jobs-output-get ${JOB_ID} fork-command-1.out
!cat fork-command-1.out

Retrieve the standard error output.

In [ ]:
!jobs-output-get ${JOB_ID} fork-command-1.err
!cat fork-command-1.err

<h3>Automating</h3>
Because we're working in Python, we can simply glue the above steps together and create a script to run jobs for us and fetch the standard output. Let's do that next.

In [ ]:
%%writefile runagavecmd.py
from setvar import *

from time import sleep

def runagavecmd(cmd,infile=None):
    setvar("REMOTE_COMMAND="+cmd)
    setvar("REQUESTBIN_URL=$(requestbin-create)")
    print("")
    print(" ** QUERY STRING FOR REQUESTBIN **")
    print('%s?inspect'%os.environ['REQUESTBIN_URL'])
    print("")
    # The input file is an optional parameter, both
    # to our function and to the Agave application.
    if infile == None:
        setvar("INPUTS={}")
    else:
        setvar('INPUTS={"datafile":"'+infile+'"}')
    setvar("JOB_FILE=job-remote-$PID.txt")
    # Create the Json for the job file.
    writefile("$JOB_FILE","""
 {
   "name":"fork-command-1",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0",
   "executionSystem": "${AGAVE_EXECUTION_SYSTEM_ID}",
   "archive": false,
   "notifications": [
    {
      "url":"${REQUESTBIN_URL}?event=\${EVENT}&jobid=\${JOB_ID}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "command":"${REMOTE_COMMAND}"
   },
   "inputs":${INPUTS}
 }""")
    # Run the job and capture the output.
    setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F $JOB_FILE)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")
    # Poll and wait for the job to finish.
    for iter in range(80): # Excessively generous
        setvar("STAT=$(jobs-status $JOB_ID)")
        stat = os.environ["STAT"]
        sleep(5.0)
        if stat == "FINISHED" or stat == "FAILED":
            break
    # Fetch the job output from the remote machine
    setvar("CMD=jobs-output-get ${JOB_ID} fork-command-1.out")
    os.system(os.environ["CMD"])
    print("All done! Output follows.")
    # Load the output into memory
    output=readfile("fork-command-1.out")
    print("=" * 70)
    print(output)

In [ ]:
import runagavecmd as r
import imp
imp.reload(r)

In [ ]:
r.runagavecmd("lscpu")

<h2>Permissions and Sharing</h3>

List the users and the permssions they have to look at the given job.

In [ ]:
!jobs-pems-list ${JOB_ID}

Grant read permission to the user, ktraxler.

In [ ]:
# permissions: READ, WRITE, READ_WRITE, ALL, NONE
!jobs-pems-update -u ktraxler -p READ ${JOB_ID}

Run the list again and see the modified result.

In [ ]:
!jobs-pems-list ${JOB_ID}

In [ ]:
!apps-pems-list ${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0

Now do the same thing for the application itself...

In [ ]:
# permissions: READ, WRITE, EXECUTE, READ_WRITE, READ_EXECUTE, WRITE_EXECUTE, ALL, and NONE
!apps-pems-update -u ktraxler -p READ_EXECUTE ${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0

In [ ]:
!apps-pems-list ${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0

## Using the TOGO web portal  

Follow the link below to run your job from a web portal.

In [ ]:
!echo http://togo.solveij.com/app/#/apps/${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0/run